<a href="https://colab.research.google.com/github/Ashu10051998/Face-Emotion-Detection/blob/main/emotion_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget https://www.dropbox.com/s/si11cws2pyho1bp/archive.zip

--2021-09-22 03:42:55--  https://www.dropbox.com/s/si11cws2pyho1bp/archive.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/si11cws2pyho1bp/archive.zip [following]
--2021-09-22 03:42:55--  https://www.dropbox.com/s/raw/si11cws2pyho1bp/archive.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc8f16331d9f49ed8962cb3caf13.dl.dropboxusercontent.com/cd/0/inline/BWlLIriq81Uu3A-C0snP2wnbdkVfGR0CGm_mzt6En4StLai-jo7mGPSLJjoTRuPOpZPTlHOHfSM2DApCCzxvKmghpTgrdS-y2YZOu-87klBKahphN5xIlByq7pTszEr2ph5_Rcfb5hFfd7xcix0yLiVG/file# [following]
--2021-09-22 03:42:55--  https://uc8f16331d9f49ed8962cb3caf13.dl.dropboxusercontent.com/cd/0/inline/BWlLIriq81Uu3A-C0snP2wnbdkVfGR0CGm_mzt6En4StLai-jo7mGPSLJjoTRuPOpZPTlHOHfSM2DApCCzxvKmghpTgrd

In [3]:
!unzip -q "/content/archive.zip"

In [4]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

tf.random.set_seed(4)

In [6]:
# Creating the Pathlib PATH objects
train_path = Path("/content/train")
test_path = Path("/content/test")

In [7]:
# Getting Image paths 
train_image_paths = list(train_path.glob("*/*"))
train_image_paths = list(map(lambda x : str(x) , train_image_paths))

train_image_paths[:10]

['/content/train/happy/Training_41477899.jpg',
 '/content/train/happy/Training_61706679.jpg',
 '/content/train/happy/Training_68886569.jpg',
 '/content/train/happy/Training_35728842.jpg',
 '/content/train/happy/Training_10717931.jpg',
 '/content/train/happy/Training_86039203.jpg',
 '/content/train/happy/Training_18383577.jpg',
 '/content/train/happy/Training_28198433.jpg',
 '/content/train/happy/Training_53772653.jpg',
 '/content/train/happy/Training_38756386.jpg']

In [8]:
# Getting their respective labels 

def get_label(image_path):
    return image_path.split("/")[-2]

train_image_labels = list(map(lambda x : get_label(x) , train_image_paths))
train_image_labels[:10]

['happy',
 'happy',
 'happy',
 'happy',
 'happy',
 'happy',
 'happy',
 'happy',
 'happy',
 'happy']

In [9]:
from sklearn.preprocessing import LabelEncoder 

In [10]:
Le = LabelEncoder()
train_image_labels = Le.fit_transform(train_image_labels)
train_image_labels[:10]

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [11]:
train_image_labels = tf.keras.utils.to_categorical(train_image_labels)

train_image_labels[:10]

array([[0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.]], dtype=float32)

In [12]:
from sklearn.model_selection import train_test_split 

Train_paths , Val_paths , Train_labels , Val_labels = train_test_split(train_image_paths , train_image_labels , test_size = 0.25)

In [13]:
classTotals = Train_labels.sum(axis=0)
classWeight = classTotals.max() / classTotals

class_weight = {e : weight for e , weight in enumerate(classWeight)}
print(class_weight)

{0: 1.810097, 1: 16.258259, 2: 1.7555124, 3: 1.0, 4: 1.4581201, 5: 1.5001385, 6: 2.268119}


In [14]:
# Function used for Transformation

def load(image , label):
    image = tf.io.read_file(image)
    image = tf.io.decode_jpeg(image , channels = 3)
    return image , label